### **Import tools**

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### **Get the data**

In [7]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type'] # tên target
data = pd.read_csv("D:\THANH\HK2\DSA\DoAnCK\project\Iris.csv", skiprows=1, header=None, names=col_names) # Lấy features từ local
data.head(10) # in 10 hàng đầu

,sepal_length,sepal_width,petal_length,petal_width,type
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa
10,4.9,3.1,1.5,0.1,Iris-setosa


### **Decision Tree** 

**Node Class**

In [8]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        #Khởi tạo node
        
        # decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # Leaf node
        self.value = value

#### **Tree class**

In [9]:
class DecisionTreeClassifier():
    def __init__(self,min_samples_split=2, max_depth=2):
        
        self.root = None #khởi tạo root
        # Điều kiện dừng 
        self.min_samples_split= min_samples_split #khi split tới khi còn số lượng sample < 2
        self.max_depth = max_depth # sâu tối đa  

    def build_tree(self, dataset, curr_depth=0):
        #đệ quy build tree
        X,Y = dataset[:,:-1], dataset[:,-1] # features lấy tất cả các cột bỏ cột cuối cùng, targets: lấy cột cuối cùng
        num_samples, num_features = np.shape(X)

        #split cho tới khi điều kiện dừng thõa mãn
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # tìm best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check nếu information gain positive 
            if best_split['info_gain'] > 0:
                # đệ quy bên trái
                left_subtree = self.build_tree(best_split["dataset_left"],curr_depth+1)
                # đệ quy bên phải
                right_subtree = self.build_tree(best_split["dataset_right"],curr_depth+1)
                # trả về Nút quyết định / Decision node
                return Node(best_split["feature_index"],best_split["threshold"],
                            left_subtree,right_subtree, best_split["info_gain"])
            #tính leaf node xem cái nào dominate
        leaf_value = self.calculate_leaf_value(Y)
            # trả về leaf node
        return Node(value=leaf_value)
    
    
    
    def get_best_split(self, dataset, num_samples, num_features):
            #tìm best split
            # dùng dictionary store best split
            best_split={}
            max_info_gain = -float("inf") # khởi tạo giá trị âm inf
            
            # loop vào tất cả features
            for feature_index in range(num_features):
                feature_values = dataset[:,feature_index] # lấy tất cả giá trị trong một cột features
                possible_thresholds= np.unique(feature_values) # lấy tất các ngưỡng giá trị khác nhau
                #thử từng ngưỡng để xác định ngưỡng tốt nhất / best split
                for threshold in possible_thresholds:
                    # tính split hiện tại
                    dataset_left, dataset_right = self.split(dataset, feature_index,threshold)
                    # check nếu các con của nó không rỗng
                    if len(dataset_left)>0 and len(dataset_right)>0:
                        y, left_y,right_y = dataset[:,-1], dataset_left[:,-1],dataset_right[:,-1]  # lấy cột target trái phải và ở nút hiện tại
                        curr_info_gain = self.information_gain(y, left_y, right_y, "gini") # tính information gain 
                        # cập nhật best split nếu cần thiết 
                        if curr_info_gain>max_info_gain:
                            best_split["feature_index"] = feature_index
                            best_split["threshold"] = threshold
                            best_split["dataset_left"]= dataset_left
                            best_split["dataset_right"]= dataset_right
                            best_split["info_gain"]= curr_info_gain
                            max_info_gain = curr_info_gain
            # trả về best split
            return best_split
    def split(self, dataset, feature_index,threshold):
            #chương trình split dataset dựa vào ngưỡng
            dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold]) 
            dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
            return dataset_left, dataset_right
    def information_gain(self,parent, l_child, r_child, mode = "entrophy"):
            #chương trình tính infomation gain 
            weight_l = len(l_child)/len(parent) # tính tỉ lệ giữa số con / số parents
            weight_r = len(r_child)/len(parent)
            if(mode == "gini"): #công thức tính gini
                gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child)+ weight_r*self.gini_index(r_child))
            else: #công thức tính entropy
                gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
            return gain
            
    def entropy(self, y):
        #tính entropy
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        #tính gini
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
    def calculate_leaf_value (self, Y):
            
            #tính xem phân loại nào ưu thế trong lá
            Y = list(Y)
            return max(Y, key=Y.count) 
    def print_tree ( self, tree=None, indent=" "):
            
            #chương trình để in cây ra
            if not tree:
                tree= self.root
            if tree.value is not None:
                print(tree.value)
            else:
                print("X_"+str(tree.feature_index), "<=", tree.threshold,"?", tree.info_gain)
                print("%sleft:" %(indent),end="")
                self.print_tree(tree.left, indent + indent)
                print("%sright:" % (indent),end="")
                self.print_tree(tree.right, indent + indent)
    def fit(self, X, Y):
        
        #train cây 
            dataset = np.concatenate((X,Y.reshape(-1,1)), axis=1)
            self.root = self.build_tree(dataset)
    def predict(self, X):
            
            #dự đoán dataset mới
            prediction = [self.make_prediction(x, self.root) for x in X]
            return prediction
    #dự đoán 1 điểm data point 
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    


### **Preprocessing**

In [10]:
Y = data["type"].values.reshape(-1,1) # lấy tất cả feature đổi thành array
#print(X.shape) 
X = data.drop("type",axis=1).values 
# chia ra hai loại samples để train và để test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2) #tỉ lệ 80/20

### **Fit The Model**

In [11]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)  
classifier.fit(X_train,Y_train)#train 
classifier.print_tree()

X_2 <= 1.9 ? 0.3415491452991453
 left:Iris-setosa
 right:X_3 <= 1.7 ? 0.40222030007360954
  left:X_2 <= 5.1 ? 0.08442269387539733
    left:X_0 <= 4.9 ? 0.04759071980963725
        left:Iris-virginica
        right:Iris-versicolor
    right:Iris-virginica
  right:X_2 <= 4.8 ? 0.026938775510204082
    left:Iris-versicolor
    right:Iris-virginica


In [12]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8666666666666667